In [ ]:
# Using PyBoruta doing feature selection 

In [1]:
import pandas as pd, numpy as np, scipy, os, itertools, sys
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
sys.path.append('..')
from func import loaders
from import_library import *

/home/john/anaconda3/envs/SCD/lib/python3.7/site-packages/pkg_resources/__init__.py:125: PkgResourcesDeprecationWarning: -VERSION-PLACEHOLDER- is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [7]:
outcome_df = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']]
data_type ='var' #mean, var
ld_excel = loaders(data_type = data_type)
ld_mat = loaders(data_type=data_type)
raw_data_path = './data/dFC matrix matlab/'
for next_path in sorted(os.listdir(raw_data_path)):
    if 'C0018FC_z.mat' not in next_path and '.mat' in next_path:
        ld_mat.load_mat(data_path = os.path.join(raw_data_path,next_path))
alff_index, alff_value, alff_columns = ld_excel.load_csv(data_path=f'./data/mdALFF matrix excel/mdALFF_{data_type}.csv')
fc_value = ld_mat.callback()

In [8]:
model_stack_ALFF = [None, None]
model_stack_FC = [None, None]
model_feature_stack = []
for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
# for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]']):
    forest = RandomForestRegressor(max_depth = 5, n_jobs=-1, random_state=120)

    # #FC feature selection
    print(f"FC {data_type} Feature Selection:", outcome_name)
    print(outcome_name)
    feat_selector_fc = BorutaPy(forest, verbose=0, random_state=120, max_iter=10)
    feat_selector_fc.fit(fc_value, np.array(outcome_df[outcome_name]))
    model_stack_FC[idx] = (feat_selector_fc)
    ranking_stack = []
    for (value, colum) in zip(feat_selector_fc.ranking_, [f'FC_{data_type}_{i}' for i in range (fc_value.shape[1])]):
        ranking_stack.append([value, colum])
    ranking_stack = sorted(ranking_stack)
    pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    pd_rs.to_csv(f'./feature_selection/feature_selection_result_pyboruta/{outcome_name}_{data_type}_fc.csv', index=False)
    model_feature_stack.append(pd_rs)

    # ALFF feature selection
    # print(f"ALFF {data_type} Feature Selection:", outcome_name)
    # feat_selector_alff = BorutaPy(forest, verbose=0, random_state=120, max_iter=10, alpha=0.01, two_step=True)
    # feat_selector_alff.fit(np.array(alff_value), np.array(outcome_df[outcome_name]))
    # model_stack_ALFF[idx] = feat_selector_alff

    # ranking_stack = []
    # for (value, colum) in zip(feat_selector_alff.ranking_,alff_value.columns.tolist()):
    #     ranking_stack.append([value, colum])
    # ranking_stack = sorted(ranking_stack)
    # pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    # pd_rs.to_csv(f'./feature_selection/feature_selection_result_pyboruta/{outcome_name}_{data_type}_alff.csv', index=False)
    # model_feature_stack.append(pd_rs)

FC var Feature Selection: AD8主觀認知障礙[0-1,>=2]
AD8主觀認知障礙[0-1,>=2]
FC var Feature Selection: MOCA客觀認知測驗分數[<=23,>=24]
MOCA客觀認知測驗分數[<=23,>=24]


In [ ]:
for i in zip(model_feature_stack, ['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
    print(i[1], data_type)
    print(i[0].head(30))
    print('\n')